In [2]:
!pip install -q \
langchain \
langchain-community \
faiss-cpu \
sentence-transformers \
transformers \
accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [3]:
import langchain
import langchain_community
import faiss
import transformers

print("Environment ready ✅")


Environment ready ✅


In [5]:
from langchain_core.documents import Document

text = """
Artificial Intelligence (AI) is the simulation of human intelligence in machines.
Machine Learning is a subset of AI that enables systems to learn from data.
Deep Learning is a subset of Machine Learning that uses neural networks.
Natural Language Processing (NLP) helps computers understand human language.
AI is used in healthcare, finance, education, and robotics.
"""

documents = [Document(page_content=text)]

print("Documents created ✅")


Documents created ✅


In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=50
)

docs = splitter.split_documents(documents)

print(f"Number of chunks: {len(docs)}")


Number of chunks: 2


In [8]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

vectorstore = FAISS.from_documents(docs, embedding_model)

print("Vector store ready ✅")


/tmp/ipython-input-973503957.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Vector store ready ✅


In [22]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

model_name = "google/flan-t5-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

def generate_answer(prompt):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            do_sample=False
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

print("Direct generation ready ✅")


Loading weights:   0%|          | 0/282 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


Direct generation ready ✅


In [11]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})


In [12]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Use the provided context to answer the question."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])


In [14]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})


In [15]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Use the provided context to answer the question."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])


In [23]:
chat_history = []

def ask_question(question):
    global chat_history

    # Retrieve relevant documents
    retrieved_docs = vectorstore.similarity_search(question, k=2)
    context = "\n\n".join([doc.page_content for doc in retrieved_docs])

    # Format history
    history_text = ""
    for role, msg in chat_history:
        if role == "human":
            history_text += f"User: {msg}\n"
        else:
            history_text += f"Assistant: {msg}\n"

    # Build structured prompt
    final_prompt = f"""
You are a helpful AI assistant.

Use ONLY the information from the provided context to answer clearly and concisely.
If the answer is not in the context, say: "I don't know based on the provided information."

Conversation History:
{history_text}

Context:
{context}

Question:
{question}

Answer:
"""

    # Generate response
    response = generate_answer(final_prompt)

    # Store memory
    chat_history.append(("human", question))
    chat_history.append(("ai", response))

    return response.strip()


In [24]:
print(ask_question("What is Artificial Intelligence?"))


the simulation of human intelligence in machines.


In [ ]:
#you can enter questions only based on the text i wrote at the start
print("Chatbot is ready! Type 'exit' to stop.\n")

while True:
    user_input = input("You: ")

    if user_input.lower() == "exit":
        print("Chat ended.")
        break

    answer = ask_question(user_input)
    print("Bot:", answer)
    print()
